In [11]:
import pandas as pd
import json
from collections import defaultdict
import re
from utilities import Tiempo
import dataExtraction
import statistics

#### Con este bucle obtenemos los datos sobre la inicialización de todos los niveles jugados

In [12]:
def extraerTiempos(rawData, minTiempoInactivo):
    #HAY QUE COMPROBAR QUE minTiempoInactivo es positivo
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erAccessed = re.compile(r'\baccessed$\b')
    erInteracted = re.compile(r'\binteracted$\b')

    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
    erSeriousGame = re.compile(r'\bserious-game$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erScreen = re.compile(r'\bscreen$\b')

    erExitGamePanel = re.compile(r'\bexit_game_panel$\b')
    erExitGamePanelCloseButton = re.compile(r'\bexit_game_panel_close_button$\b')
    
    data = defaultdict(defaultdict)
    infoNivel = defaultdict(defaultdict)
    ultimaTrazaRegistrada = defaultdict()
    tiemposInactivo = defaultdict(list)
    inicioYFinJuego = defaultdict(list)
    intentosNivel = defaultdict(defaultdict)
    anomalias = []
    
    for evento in rawData:         
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
        objectId = evento["object"]["id"]
        
        if erLevel.search(obj): #Si el objeto de la acción es un nivel
            levelCode = erIdLevel.split(objectId)[-1]
            if erInitialized.search(verb): #Si la acción es inicio o reinicio
                if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                    if levelCode in data[name]:
                        data[name][levelCode].append({"ini" : timestamp, "fin" : None, "stars" : ""})
                    else:
                        data[name][levelCode] = [{"ini" : timestamp, "fin" : None, "stars" : ""}]
                else:
                    if levelCode in intentosNivel[name]:
                        intentosNivel[name][levelCode]["intentos"] += 1
                    else:
                        intentosNivel[name][levelCode] = {"intentos" : 1}
            elif erCompleted.search(verb):
                if evento["result"]["score"]["raw"] > 0 :
                    dataExtraction.anadirAlDiccionario(infoNivel, evento, name, levelCode)
                    if levelCode in data[name]:
                        data[name][levelCode][-1]["fin"] = timestamp
                        data[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
                    else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                        anomalias.append(evento)
                else:
                    if not levelCode in data[name]:
                         anomalias.append(evento) 
        
        #Algunos jugadores cierran el juego el juego con Alt+F4 o dando a la x de la ventana, esto no queda registrado
        #Para poder tener esto en cuenta habría que recorrer los datos 2 veces ya que no podemos quedarnos con el último 
        #valor antes del initialized porque los timestamps no vienen bien ordenados
        
        elif erSeriousGame.search(obj) and erInitialized.search(verb): #Nos indica que el jugador ha abierto el juego
            inicioYFinJuego[name].append({"inicio" : timestamp, "fin" : None})
        
        """ Intento de obtener el progreso de los jugadores
        if erProgressed.search(verb):
            progresoActual = evento["result"]["extensions"]["https://w3id.org/xapi/seriousgames/extensions/progress"]
            if name in ultimoNivelCompletado:
                if progresoActual > ultimoNivelCompletado[name]["progreso"]:
                    ultimoNivelCompletado[name] = {"nivel" : None, "progreso" : progresoActual}
            else:
                ultimoNivelCompletado[name] = {"nivel" : None, "progreso" : progresoActual}"""
      
    for evento in rawData:
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
                
        for i in range(len(inicioYFinJuego[name])):
            if (i + 1) < len(inicioYFinJuego[name]): #No estamos en la ultima partida del jugador
                if Tiempo(timestamp, inicioYFinJuego[name][i+1]["inicio"]) > Tiempo("1s"): #No es una trazas que aparece antes de iniciar el juego
                    inicioYFinJuego[name][i]["fin"] = timestamp
            else:
                inicioYFinJuego[name][i]["fin"] = timestamp
        if name in ultimaTrazaRegistrada:
                t = Tiempo(ultimaTrazaRegistrada[name], timestamp)
                if t > minTiempoInactivo:
                    tiemposInactivo[name].append(t)
                   
                    for ini in inicioYFinJuego[name]:
                        if(Tiempo(ini["inicio"], timestamp) < Tiempo("1s")):
                            del tiemposInactivo[name][-1] #Eliminamos los tiempos inactivos que sean de distintos inicios del juego
                            break
                        
        ultimaTrazaRegistrada[name] = timestamp
                    
                    
    return {"tiemposNivel" : data, "tiempoTotal" : inicioYFinJuego, "anomalias" : anomalias, "tiemposInactivo" : tiemposInactivo, "infoNivel" : infoNivel, "intentosNivel" : intentosNivel}              

In [13]:
def extraerTiemposEntreInteracciones(rawData):
    tiempoInteracciones = defaultdict()
    ultimaInteraccionJugador = defaultdict()
    
    erInteracted = re.compile(r'\binteracted$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erLevelExitButton = re.compile(r'\blevel_exit_button$\b') 
    erInitialized = re.compile(r'\binitialized$\b')
    erSeriousGame = re.compile(r'\bserious-game$\b')    
    erCompleted = re.compile(r'\bcompleted$\b')
    erLevel = re.compile(r'\blevel$\b')
        
    for e in rawData:
        verb = e["verb"]["id"]
        obj = e["object"]["definition"]["type"]
        objectId = e["object"]["id"]
        name = e["actor"]["name"]
        
        if erSeriousGame.search(obj) and erInitialized.search(verb): #Si inicia el juego se reinicia su ultima interaccion
            if name in ultimaInteraccionJugador:
                del ultimaInteraccionJugador[name]
                
        elif erLevel.search(obj) and erCompleted.search(verb): #Si completa un nivel se reinicia su ultima interaccion
            if name in ultimaInteraccionJugador:
                del ultimaInteraccionJugador[name]
        
        elif erGameObject.search(obj) and erInteracted.search(verb):
            if "result" in e and "extensions" in e["result"] and "level" in e["result"]["extensions"]:
                levelCode = e["result"]["extensions"]["level"]
                if not erLevelExitButton.search(objectId) and levelCode != "editor_level":
                    timestamp = e["timestamp"]
                    if name in ultimaInteraccionJugador:
                        #e["result"]["extensions"]["action"] <-- Que accion realiza
                        if levelCode in tiempoInteracciones:
                            t = abs(Tiempo(ultimaInteraccionJugador[name], timestamp))
                            tiempoInteracciones[levelCode].append(t)
                            #if t > 600:
                                #print(name)
                                #print(ultimaInteraccionJugador[name])
                                #print(timestamp)
                                #print("--------------------------------")
                        else:
                            tiempoInteracciones[levelCode] = [Tiempo(ultimaInteraccionJugador[name], timestamp)]
                    
                    ultimaInteraccionJugador[name] = timestamp
    return tiempoInteracciones

In [14]:
def tiempoTotalJuego(inicioYFinJuego):
    tiempoTotal = defaultdict()
    for player in inicioYFinJuego:
        for time in inicioYFinJuego[player]:
            if time["fin"] != None:
                if player in tiempoTotal:
                    tiempoTotal[player] += Tiempo(time["inicio"], time["fin"])
                else:
                    tiempoTotal[player] = Tiempo(time["inicio"], time["fin"])
    return tiempoTotal

In [15]:
def tiempoPorNiveles(data, anomalias):
    tiemposJugados = defaultdict(defaultdict)
    for player in data:
        for level in data[player]:
            for times in data[player][level]:
                if times["fin"] != None: #Si no se aborto el intento del nivel
                    try:
                        timeDifference = Tiempo(times["ini"], times["fin"])
                        if level in tiemposJugados[player]:
                            tiemposJugados[player][level].append({"time" : timeDifference, "stars" : times["stars"]})
                        else:
                            tiemposJugados[player][level] = [{"time" : timeDifference, "stars" : times["stars"]}]
                    except ValueError: #Algunos timestamps de finalizacion estan vacios, ¿Tiene que ser asi?
                        anomalias.append({"jugador" : player,
                                  "nivel" : level,
                                  "Timestamp-Inicio" : times["ini"],
                                  "Timestamp-Fin" : times["fin"],
                                  "Descripcion:" : "No se ha podido parsear uno de los 2 timestamps"})
    return tiemposJugados

In [16]:
def buscarTarjetas(rawData):
    data = []
    erIdLevel = re.compile(r'/')
    for evento in rawData:
        idJugador = evento["actor"]["name"]
        levelCode = erIdLevel.split(evento["object"]["id"])[-1]
        if idJugador == "bdwyj" and "result" in evento and "extensions" in evento["result"] and "level" in evento["result"]["extensions"]:
            if evento["result"]["extensions"]["level"] == "variables_9":
                data.append(evento)
        elif idJugador == "bdwyj" and levelCode == "variables_9":
            data.append(evento)
    return data

In [17]:
def getMediaTiemposYEstrellas(data):
    tiemposYEstrellasMedios = defaultdict()
    sumasTotales = defaultdict()
    for user in data:
        for level in data[user]:
            for time in data[user][level]:
                if level in tiemposYEstrellasMedios:
                    tiemposYEstrellasMedios[level]["tiempo"] += time["time"]
                    tiemposYEstrellasMedios[level]["stars"] += time["stars"]
                    sumasTotales[level] += 1
                else:
                    tiemposYEstrellasMedios[level] = {"tiempo" : time["time"], "stars" : time["stars"]}
                    sumasTotales[level] = 1
    for level in tiemposYEstrellasMedios:
        tiemposYEstrellasMedios[level]["tiempo"] = tiemposYEstrellasMedios[level]["tiempo"]/sumasTotales[level];
        tiemposYEstrellasMedios[level]["stars"] = round(tiemposYEstrellasMedios[level]["stars"]/sumasTotales[level], 2);
    return tiemposYEstrellasMedios

In [18]:
def getDesempenioJugadores(tiempos, tiemposMedios, porcDebajo, porcEncima, primerIntento = True):
    #tiempos -> diccionario con los tiempos de cada jugador en cada nivel
    #tiemporMedios -> media de los tiempos por nivel
    #porcDebajo -> porcentaje que se usara para filtrar a los jugadores con un tiempo igual o inferior a este
    #porcEncima -> porcentaje que se usara para filtrar a los jugadores con un tiempo igual o superior a este
    #primerIntento -> true cuando solo se quiere tener en cuenta el primer intento del jugador, false cuando no
    jugadoresPorDebajo = defaultdict(defaultdict) #Jugadores con buen desempeño, menor tiempo que media
    jugadoresPorEncima = defaultdict(defaultdict) #Jugaodre con mal desempeño, mayor tiempo que media

    for player in tiempos:
        for level in tiempos[player]:
            for time in tiempos[player][level]:
                if time["time"] <= (tiemposMedios[level]["tiempo"]*(1-porcDebajo)):
                    if level in jugadoresPorDebajo[player]:
                        jugadoresPorDebajo[player][level].append(time["time"])
                    else:
                        jugadoresPorDebajo[player][level] = [time["time"]]
                        
                elif time["time"] >= (tiemposMedios[level]["tiempo"]*(1+porcEncima)):
                    if level in jugadoresPorEncima[player]:
                        jugadoresPorEncima[player][level].append(time["time"])
                    else:
                        jugadoresPorEncima[player][level] = [time["time"]]
                        
                if primerIntento:
                    break
                    
    return jugadoresPorDebajo, jugadoresPorEncima

In [19]:
def getUltimoNivelSuperado (jugadores, levels, tiemposJ):
    ultimoNivel = defaultdict()
    for jugador in jugadores:
        for l in levels:
            if l in tiemposJ[jugador]:
                ultimoNivel[jugador] = l
    return ultimoNivel

In [20]:
def maxMinTiempoPorNivel(tiemposJ):
    tiempos = defaultdict()

    for jugador in tiemposJ:
        for nivel in tiemposJ[jugador]:
            for tiempo in tiemposJ[jugador][nivel]:
                time = Tiempo(str(tiempo["time"]))
                if not nivel in tiempos:
                    tiempos[nivel] = {"max" : time, "min" : time}
                elif time > tiempos[nivel]["max"]:
                    tiempos[nivel]["max"] = time
                elif time < tiempos[nivel]["min"]:
                    tiempos[nivel]["min"] = time
    return tiempos;

In [21]:
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None

In [24]:
JSONFile = open('trazasOrdenadas.json')
rawData = json.load(JSONFile)
minTiempoInactivo = Tiempo("1m/30s")
result = extraerTiempos(rawData, minTiempoInactivo)
#result = extraerTiemposEntreInteracciones(rawData)
JSONFile.close()
#pd.DataFrame(result["data"])
#pd.DataFrame.from_dict(result["tiempoTotal"], orient='index').transpose()
#pd.DataFrame.from_dict(result["tiemposInactivo"], orient='index').transpose()
#pd.DataFrame(result["infoNivel"])
#pd.DataFrame(result["intentosNivel"])
#print(result)

In [ ]:
#Numero de jugadores
len(result["intentosNivel"].keys())

In [ ]:
tiemposJ = tiempoPorNiveles(result["tiemposNivel"], result["anomalias"])
pd.DataFrame(tiemposJ)

In [ ]:
tiemposMedios = getMediaTiemposYEstrellas(tiemposJ)
pd.DataFrame(tiemposMedios)

In [ ]:
#Nivel mas avanzado completado
ultimoNivel = getUltimoNivelSuperado(tiemposJ.keys(), tiemposMedios.keys(), tiemposJ)
pd.DataFrame.from_dict(ultimoNivel, orient='index').transpose()

In [ ]:
#Tiempo mínimo y máximo de cada nivel
tiemposMinMax = maxMinTiempoPorNivel(tiemposJ)
pd.DataFrame(tiemposMinMax)

In [ ]:
porcentajePorDebajo = 0.2
porcentajePorEncima = 0.5
jPorDebajo, jPorEncima = getDesempenioJugadores(tiemposJ, tiemposMedios, porcentajePorDebajo, porcentajePorEncima)
pd.DataFrame(jPorEncima)

In [25]:
tiempoTJuego = tiempoTotalJuego(result["tiempoTotal"])
pd.DataFrame(tiempoTJuego, index = ["Tiempo"])
#Los tiempos del jugador knewj salen mal porque las trazas tienen mal puesto el tiempo

,gqoj,hgfx,gton,uala,pbgky,auvqa,qeqyj,fkmtp,hjpbw,eprdu,bdwyj,ftpnb,ikcar,gleqe,gcpze,ejeuw,loemd,srjon,sxipu,uzzpo,vsxld,khadt,zxvkj,kmqou,xnpqf,epqkm,ipdcp,zwzfv,krahz,rdiaf,grkij,zcibf,yvpmf,pygoc,ubmtx,swvxx,edxmq,dwqwi,kkoif,asopk,dunul,onyep,cmbfi,bhyhw,oahey,accul,tekwp,yfohf,nmveq,tjfzy,sjpkc,camhi,stiry,ewqbs,fivin,lwryl,psido,gdljy,tinco,zbgwk,mymho,fyyya,ucdvy,yxewz,wjaxi,dvmlu,mjiso,gguuc,hpdxj,tgwmn,kkliq,dmwvy,hqmhq,rziia,ztyed,yxjji,vddlg,yazod,qfsdj,idmzi,bjvnt,atlzs,ngavz,tqgyp,vvqlm,ahxym,tavin,aotlm,lhrbj,zdnso,mgaak,knewj,huyyb,wqmgq,rqlug,tvxow,jhbuv,xtdrq,jozyx,pzfdd,ptmkr,qwnpf,buvgs,nuasx,pbwkf,thaho,mewpz,fmtrq,icllt,hcbyl,yyvas,nhbnj,rqaby,itkas,khfha,gzgfk,sbexy,rkmrw,uniee,esrit,kzles,nplrf,qgcgc,ixrux,pqwaf,rfofs,cojqb,lohyo,tflzv,qbsjv,rsjxu,sybwc,blvkx,raqdt,tcvyz,fbldj,bajbm,eayrh,orjyn,prpej,fmupf,xbmmw,vkfnt,pleui,olugf,epjcx
Tiempo,27m/24s,3m/51s,16s,3m/7s,1h/29m/54s,1h/27m/7s,1h/25m/21s,1h/26m/13s,1h/26m/31s,1h/28m/12s,1h/28m/32s,1h/24m/23s,1h/21m/37s,1h/28m/42s,1h/22m/13s,1h/19m/49s,1h/27m/43s,1h/24m/30s,1h/6m/33s,1h/18m/49s,1h/23m/19s,1h/21m/55s,1h/18m/33s,1h/19m/35s,1h/25m/48s,37m/52s,1h/21m/46s,1h/22m/3s,1h/32m/34s,1h/33m/23s,1h/30m/57s,1h/31m/9s,1h/26m/31s,1h/33m/8s,1h/28m/13s,1h/27m/58s,1h/29m/56s,1h/30m/11s,1h/30m/46s,1h/31m/9s,1h/30m/34s,1h/30m/12s,1h/29m/42s,1h/30m/18s,1h/32m/1s,1h/28m/26s,1h/26m/12s,1h/30m/19s,1h/22m/37s,1h/31m/17s,1h/27m/54s,1h/16m/37s,58m/25s,1h/14m/28s,1h/11m/3s,1h/9m/35s,1h/18m/24s,1h/12m/17s,1h/9m/29s,1h/10m/59s,1h/10m/40s,1h/19m/19s,1h/15m/50s,1h/15m/44s,1h/15m/10s,1h/18m/48s,1h/19m/57s,1h/17m/56s,1h/18m/58s,1h/18m/2s,1h/16m/52s,1h/18m/41s,1h/14m/43s,1h/16m/24s,1h/18m/16s,1h/15m/4s,1h/21m,1h/17m/37s,1h/17m/27s,1h/18m/24s,1h/20m/36s,1h/23m/42s,1h/19m/48s,1h/19m/53s,1h/17m/4s,1h/21m/55s,1h/15m/23s,1h/17m/8s,1h/22m/50s,1h/22m/29s,1h/22m/59s,69h/38m/55s,1h/17m/26s,1h/14m/47s,1h/20m/17s,1h/18m/36s,1h/22m,1h/18m/53s,1h/17m/57s,1h/16m/20s,1h/18m/59s,1h/17m,1h/21m/37s,1h/15m/37s,1h/25m/44s,1h/24m/49s,1h/19m/43s,1h/23m/30s,1h/22m/21s,1h/26m/43s,1h/25m/18s,1h/16m/27s,1h/15m/33s,1h/15m/43s,1h/23m/42s,33m/35s,1h/18m/23s,1h/25m,1h/21m/57s,1h/23m/25s,1h/19m/54s,1h/13m/12s,1h/31m/32s,1h/32m/42s,1h/32m/16s,1h/13m/24s,1h/28m/6s,1h/23m/27s,1h/29m/23s,1h/28m/40s,1h/29m/47s,1h/31m/48s,1h/27m/1s,1h/28m/57s,1h/29m/22s,1h/28m/36s,1h/33m/31s,1h/33m/24s,1h/32m/30s,1h/24m/40s,1h/26m/42s,1h/30m/19s,1h/22m/25s,1h/29m/56s,1h/18m/35s,38m/14s


In [ ]:
Tiempo("2022-05-18T07:13:48.865Z", "2022-05-18T06:13:48.865Z")

In [ ]:
Tiempo(str(statistics.mean([int(Tiempo("0s")), int(Tiempo("1h")), int(Tiempo("2h"))])) + "s")

In [ ]:
abs(Tiempo("-2h-1s"))

In [ ]:
datos = pd.read_json('traces_Articoding_Escolapias.json')
lista = datos.values.tolist()
cont = 0
anterior = lista[cont]
for e in lista:
    if(anterior[timestamp] > e[timestamp]):
        print(anterior)
        print(e)
        print("---------")
    cont += 1
    anterior = lista[cont]

In [10]:
def myFunc(e):
    return e['timestamp']

JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)

rawData.sort(key = myFunc)
jsonString = json.dumps(rawData)

jsonFile = open("trazasOrdenadas.json", "w")
jsonFile.write(jsonString)
jsonFile.close()